# Linear Algebra - cont'd

In [ ]:
import numpy as np
import scipy.linalg as la

### Iterative methods:

**Jacobi method:**

A system of linear algebraic equations $ \mathbb{A} x = b $ with [diagonally dominant](https://en.wikipedia.org/wiki/Diagonally_dominant_matrix) coefficient matrix $ \mathbb{A} $ can be solved iteratively using [Jacobi method](https://en.wikipedia.org/wiki/Jacobi_method):

$$
x_i^{(k+1)} = \frac{1}{a_{ii}} \left( b_i - \sum_{j = 1}^{i-1} a_{ij} x_j^{(k)} - \sum_{j = i + 1}^{n} a_{ij} x_j^{(k)} \right), \qquad i = 1, 2, \dots, n.
$$

**Example:** Implement the Jacobi method:

In [ ]:
def jacobi_method(A, b, precision):
    """
    Solves system of linear equations iteratively using Jacobi's algorithm.
    Args:
        A (array_like): A n-by-n diagonally dominant matrix
        b (array_like): RHS vector of size n
        precision (float): Error tolerance
    Returns:
        numpy.ndarray: Vector of solution
    """

    # add your code here

Compare the function above with the `scipy.linalg.solve` function:

In [ ]:
A = np.random.rand(3, 3) + 10. * np.eye(3) # create diagonally dominant matrix to ensure convergence
b = np.random.rand(3)
np.testing.assert_almost_equal(jacobi_method(A, b, 1.0e-15), la.solve(A, b), decimal=15)

**Gauss-Seidel method:**

A system of linear algebraic equations $ \mathbb{A} x = b $ with [diagonally dominant](https://en.wikipedia.org/wiki/Diagonally_dominant_matrix) coefficient matrix $ \mathbb{A} $ can be solved iteratively using [Gauss-Seidel method](https://en.wikipedia.org/wiki/Gauss%E2%80%93Seidel_method):

$$
x_i^{(k+1)} = \frac{1}{a_{ii}} \left( b_i - \sum_{j = 1}^{i-1} a_{ij} x_j^{(k+1)} - \sum_{j = i + 1}^{n} a_{ij} x_j^{(k)} \right), \qquad i = 1, 2, \dots, n.
$$

**Example:** Implement the Gauss-Seidel method:

In [ ]:
def gauss_seidel_method(A, b, precision):
    """
    Solves system of linear equations iteratively using Gauss-Seidel's algorithm.
    Args:
        A (array_like): A n-by-n diagonally dominant matrix
        b (array_like): RHS vector of size n
        precision (float): Error tolerance
    Returns:
        numpy.ndarray: Vector of solution
    """

    # add your code here

Compare the function above with the `scipy.linalg.solve` function:

In [ ]:
A = np.random.rand(3, 3) + 10. * np.eye(3) # create diagonally dominant matrix to ensure convergence
b = np.random.rand(3)
np.testing.assert_almost_equal(gauss_seidel_method(A, b, 1.0e-15), la.solve(A, b), decimal=15)

**Successive over-relaxation method:**

A system of linear algebraic equations $ \mathbb{A} x = b $ can be solved iteratively using [successive over-relaxation](https://en.wikipedia.org/wiki/Successive_over-relaxation) method:

$$
x_i^{(k+1)} = (1 - \omega) x_i^{(k)}  + \frac{\omega}{a_{ii}} \left( b_i - \sum_{j = 1}^{i-1} a_{ij} x_j^{(k+1)} - \sum_{j = i + 1}^{n} a_{ij} x_j^{(k)} \right), \qquad i = 1, 2, \dots, n.
$$

The optimal value of $ \omega $ can be found as

$$
\omega_{opt} = \frac{2}{1 + \sqrt{1 - \rho^2 \left( \mathbb{B} \right)}}, \qquad \mathbb{B} = -(\mathbb{D} + \mathbb{L})^{-1} \mathbb{U}
$$

**Example:** Implement the successive over-relaxation (SOR) method:

In [ ]:
def successive_overrelaxation_method(A, b, omega, precision):
    """
    Solves system of linear equations iteratively using successive over-relaxation (SOR) method.
    Args:
        A (array_like): A n-by-n matrix
        b (array_like): RHS vector of size n
        omega (float): Relaxation factor
        precision (float): Error tolerance
    Returns:
        numpy.ndarray: Vector of solution
    """

    # add your code here

Compare the function above with the `scipy.linalg.solve` function:

In [ ]:
A = np.random.rand(3, 3) + 10. * np.eye(3) # create diagonally dominant matrix to ensure convergence
b = np.random.rand(3)

L = np.tril(A, -1) # get lower triangular matrix with zeros on diagonal
U = np.triu(A, 1) # get upper triangular matrix with zeros on diagonal
D = A - L - U # get diagonal matrix
B = -np.dot(la.inv(D + L), U) # calculate iteration matrix
rho = np.max(np.abs(la.eigvals(B))) # find spectral radius (i.e. maximal eigenvalue in absolute value)
omega = 2.0 / (1.0 + np.sqrt(1.0 - rho**2)) # find optimal relaxation factor

np.testing.assert_almost_equal(successive_overrelaxation_method(A, b, omega, 1.0e-15), la.solve(A, b), decimal=15)

### Eigenvalues:

**Example:** Implement the power iteration method:

In [ ]:
def power_iteration(A, max_it=500):
    """
    Finds the greatest eigen value (in absolute value) of given matrix and its corresponding eigenvector.
    Args:
        A (array_like): A n-by-n diagonalizable matrix
        max_it (int): Maximum number of iterations
    Returns:
        numpy.ndarray: Eigenvector corresponding to a greatest eigenvalue (in absolute value)
        float: Greatest eigenvalue (in absolute value)
    """

    # add your code here

In [ ]:
# test of the power_iteration function
A = np.random.rand(3, 3)
e_vec, e_val = power_iteration(A, 20)
print(e_val)
print(np.max(np.abs(la.eigvals(A))))

### Gradient methods:

In [ ]:
def conjugate_gradient_method(A, b, x, eps=1.0e-10):
    """
    Solves system of linear equations using conjugate gradient method.
    Args:
        A (array_like): A n-by-n regular matrix
        b (array_like): RHS vector of size n
        x (array_like): Initial guess
        eps (float): Error tolerance
    Returns:
        numpy.ndarray: Vector of solution
    """

    # add your code here

In [ ]:
# test of the conjugate_gradient_method function

n = 3 # size of the problem
A = np.random.rand(n, n)
A = np.tril(A) + np.tril(A, -1).T # generate random symmetric matrix (should check also wheter the matrix is positive-definite)
b = np.random.rand(n) # random RHS vector
x_0 = np.zeros(len(b)) # initial guess

print(conjugate_gradient_method(A, b, x_0))
print(la.solve(A, b))